<h2 id="exercises-i">Exercises I</h2>
<p>Run <code>python -m pip install pymysql</code> from your terminal to install pymysql.</p>
<p>Create a notebook or python script named <code>advanced_dataframes</code> to do your work in for these exercises.</p>


1. Run <code>python -m pip install pymysql</code> from your terminal to install the mysql client (any folder is fine)</li>

2. cd into your exercises folder for this module and run <code>echo env.py &gt;&gt; .gitignore</code></li>

3. <p>Create a function named <code>get_db_url</code>. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.</p>

In [1]:
import pandas as pd
import numpy as np

In [33]:
def get_db_url(db_name):
    from env import user, host, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

4. <p>Use your function to obtain a connection to the <code>employees</code> database.</p>

In [3]:
url = get_db_url('employees')

In [4]:
employees = pd.read_sql('SELECT * FROM employees', url)
employees

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


5. <p>Once you have successfully run a query:</p>
<p>a. Intentionally make a typo in the database url. What kind of error message do you see?</p>

<p>b. Intentionally make an error in your SQL query. What does the error message look like?</p>

In [ ]:
pd.read_sql('SELECT * FROM emploees LIMIT 5 OFFSET 50', url)

6. <p>Read the <code>employees</code> and <code>titles</code> tables into two separate DataFrames.</p>

In [ ]:
employees = pd.read_sql('SELECT * FROM employees', url)

In [ ]:
employees

In [5]:
employees.shape

(300024, 6)

In [6]:
def get_db_url(db_name):
    from env import user, host, password
    return f'mysql+pymysql://{user}:{password}@{host}/titles'

In [7]:
url2 = get_db_url('titles')

7. <p>How many rows and columns do you have in each DataFrame? Is that what you expected?</p>

In [8]:
query = '''
SELECT
    t.title as title,
    d.dept_name as dept_name,
    t.to_date as to_date,
    t.from_date as from_date
FROM titles t
JOIN dept_emp USING (emp_no)
JOIN departments d USING (dept_no)
'''

title_dept = pd.read_sql(query, url)
title_dept.head()

,title,dept_name,to_date,from_date
0,Staff,Customer Service,1996-11-09,1990-01-22
1,Senior Staff,Customer Service,9999-01-01,1996-09-20
2,Staff,Customer Service,1996-09-20,1989-09-20
3,Senior Staff,Customer Service,9999-01-01,2000-05-04
4,Staff,Customer Service,2000-05-04,1992-05-04


8. <p>Display the summary statistics for each DataFrame.</p>


In [ ]:
employees.describe()

In [9]:
title_dept.shape

(489903, 4)

In [10]:
employees.shape

(300024, 6)

In [11]:
title_dept.describe()

,title,dept_name,to_date,from_date
count,489903,489903,489903,489903
unique,7,9,5888,6393
top,Engineer,Development,9999-01-01,1996-04-19
freq,126984,125586,265332,148


9. <p>How many unique titles are in the <code>titles</code> DataFrame?</p>

In [ ]:
# 7

10. <p>What is the oldest date in the <code>to_date</code> column? </p>

In [12]:
title_dept.sort_values(by = 'to_date', ascending = True).head(1)

,title,dept_name,to_date,from_date
352131,Engineer,Quality Management,1985-03-01,1985-02-17


In [13]:
title_dept[title_dept.to_date == title_dept.to_date.min()]

,title,dept_name,to_date,from_date
352131,Engineer,Quality Management,1985-03-01,1985-02-17


11. <p>What is the most recent date in the <code>to_date</code> column?</p>

In [14]:
title_dept.sort_values(by = 'to_date', ascending = False)

,title,dept_name,to_date,from_date
244951,Technique Leader,Production,9999-01-01,1997-12-18
229979,Senior Staff,Marketing,9999-01-01,1995-12-31
229990,Senior Staff,Marketing,9999-01-01,2001-09-20
229989,Staff,Marketing,9999-01-01,1994-03-02
229987,Senior Staff,Marketing,9999-01-01,1997-03-31
...,...,...,...,...
203490,Staff,Human Resources,1985-04-17,1985-04-14
45150,Senior Engineer,Development,1985-03-23,1985-02-10
405037,Staff,Research,1985-03-20,1985-02-26
150254,Engineer,Development,1985-03-11,1985-02-10


In [15]:
title_dept[title_dept.to_date == title_dept.to_date.max()]

,title,dept_name,to_date,from_date
1,Senior Staff,Customer Service,9999-01-01,1996-09-20
3,Senior Staff,Customer Service,9999-01-01,2000-05-04
5,Senior Staff,Customer Service,9999-01-01,1996-05-28
7,Senior Staff,Customer Service,9999-01-01,1993-09-02
11,Staff,Customer Service,9999-01-01,1998-05-01
...,...,...,...,...
489892,Senior Staff,Sales,9999-01-01,1990-06-13
489895,Senior Staff,Sales,9999-01-01,1988-12-26
489896,Senior Staff,Sales,9999-01-01,1999-02-11
489898,Senior Staff,Sales,9999-01-01,1992-08-11


In [20]:
url = get_db_url("employees")
sql = '''
SELECT * FROM titles where to_date != '9999-01-01'
'''
titles2 = pd.read_sql(sql, url)
titles2.head()

,emp_no,title,from_date,to_date
0,10004,Engineer,1986-12-01,1995-12-01
1,10005,Staff,1989-09-12,1996-09-12
2,10007,Staff,1989-02-10,1996-02-11
3,10008,Assistant Engineer,1998-03-11,2000-07-31
4,10009,Assistant Engineer,1985-02-18,1990-02-18


In [16]:
title_dept[(title_dept.to_date.astype(str) < '9999-01-01')].max()

title        Technique Leader
dept_name               Sales
to_date            2002-08-01
from_date          2002-06-30
dtype: object

In [22]:
titles2.to_date.max()

datetime.date(2002, 8, 1)

In [25]:
titles2.to_date = pd.to_datetime(titles2.to_date)

In [28]:
titles2.sort_values(by = 'to_date')

,emp_no,title,from_date,to_date
7303,20869,Engineer,1985-02-17,1985-03-01
186225,474918,Engineer,1985-02-10,1985-03-11
158686,434232,Staff,1985-02-26,1985-03-20
16209,34059,Senior Engineer,1985-02-10,1985-03-23
127199,287732,Staff,1985-04-14,1985-04-17
...,...,...,...,...
95874,241410,Senior Engineer,2001-04-20,2002-08-01
65708,106707,Staff,1997-08-01,2002-08-01
119779,276731,Senior Engineer,1998-01-09,2002-08-01
108052,259430,Engineer,1993-08-01,2002-08-01
